# Run: Push for 1M 2: The pushening

```
Lead     : ababaian / jt
Issue    : 
Version  : v0.3.3
start    : 2020 06 20
complete : 2020 06 20
files    : ~/serratus/notebook/200620_ab/
s3_files : s3://serratus-public/notebook/200620_ab//
output   : s3://serratus-public/out/200620_inv/
```

### Objectives
- Reach 1M accessions / 24 hours


## SRA Access - Batman returns (Chiroptera)

Search:
```
txid9397[Organism:exp] AND "platform illumina"[Properties]
```

Accessed: `2020/06/20`

Results : `2716` saved as `bat_SraRunInfo.csv`


### Initialize local workspace

In [1]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS

# Create local run directory
WORK="$SERRATUS/notebook/200620_ab"
mkdir -p $WORK; cd $WORK

# S3 notebook path
S3_WORK='s3://serratus-public/notebook/200620_ab/'

# date and version
date
git rev-parse HEAD # commit version

Sat Jun 20 18:30:15 PDT 2020
0dec6931fdbfd5fef9be9fadc1a9c82665ef0fc9


## SRA Accession Initialization



### Remove previously completed accessions

In [4]:
cd $WORK

# Get previous SraRunInfo.csv files
ln -s ../200613_ab/invert_SraRunInfo.csv ./
ln -s ../200613_ab/mu_SraRunInfo.csv     ./
ln -s ../200612_ab/v2.sra.complete       ./

In [5]:
# Download a list of run completed last weekend
aws s3 ls s3://serratus-public/out/200613_inv/summary/ > inv13.sra
aws s3 ls s3://serratus-public/out/200613_mu/summary/  > mu13.sra


In [6]:
cat inv13.sra mu13.sra \
  | sed 's/^...............................//g' - \
  | cut -f1 -d'.' - > invmu.sra.complete

In [7]:
wc -l *sra.complete

cat v2.sra.complete invmu.sra.complete > v3.sra.complete

 1337391 invmu.sra.complete
 1147209 v2.sra.complete
 2484600 total


In [11]:
head -n1 mu_SraRunInfo.csv > sra.header
grep -vif v3.sra.complete bat_SraRunInfo.csv    | tail -n+2 - > v3_bat_SraRunInfo.csv
grep -vif v3.sra.complete mu_SraRunInfo.csv     | tail -n+2 - > v3_mu_SraRunInfo.csv
grep -vif v3.sra.complete invert_SraRunInfo.csv | tail -n+2 - > v3_inv_SraRunInfo.csv

cat sra.header v3_*_Sra* > v3_SraRunInfo.csv

wc -l *
md5sum *

      2823 bat_SraRunInfo.csv
    828553 inv13.sra
   2193741 invert_SraRunInfo.csv
   1337391 invmu.sra.complete
    508838 mu13.sra
    890747 mu_SraRunInfo.csv
         1 sra.header
   1147209 v2.sra.complete
   2484600 v3.sra.complete
   1355322 v3_SraRunInfo.csv
      1862 v3_bat_SraRunInfo.csv
   1042607 v3_inv_SraRunInfo.csv
    310852 v3_mu_SraRunInfo.csv
  12104546 total
1108e9cda3e07b55b19ece9ee8ac4dca  bat_SraRunInfo.csv
80541d76ca890aeced88ce85ecbb006f  inv13.sra
ccd2bc301495cddf11a95e63e746ce8f  invert_SraRunInfo.csv
2a2135e9b6fad700b312fac9dca90d1b  invmu.sra.complete
f41f63f1efacee92454d45583084b0f1  mu13.sra
a9e14f6043f70e485ebebeb81ace8da7  mu_SraRunInfo.csv
0a9601837e2e43ec3cc4472943eea408  sra.header
0b4dba4410ffe8b72b99a170e3a757b4  v2.sra.complete
ba4a93f658a13c8c962926c51b40127e  v3.sra.complete
78f41e69162f7efb55d268e11a3dcb88  v3_SraRunInfo.csv
bcc12969571f3d839a792014c35db118  v3_bat_SraRunInfo.csv
9d5404b01e240786c52f0777459b0bb2  v3_inv_SraRunInfo.csv
680434c

In [13]:
tail -n+2 v3_SraRunInfo.csv \
  | split -l 300000 - 

In [16]:
cat sra.header xaa > v3_batch1.csv
cat sra.header xab > v3_batch2.csv
cat sra.header xac > v3_batch3.csv
cat sra.header xad > v3_batch4.csv
cat sra.header xae > v3_batch5.csv

rm xa*

### Terraform Initialize 1

In [19]:
# For rapid batching; copy out serratus folder
SERRATUS=/home/artem/Desktop/run/serratus1
TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..a4fce76 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -92,7 +92,7 @@ module "scheduler" {
   
   security_group_ids = [aws_security_group.internal.id]
   key_name           = var.key_name
-  instance_type      = "c5.large"
+  instance_type      = "r5.4xlarge"
   dockerhub_account  = var.dockerhub_account
   scheduler_port     = var.scheduler_port
 }
@@ -105,7 +105,7 @@ module "monitoring" {
   key_name           = var.key_name
   scheduler_ip       = module.scheduler.private_ip
   dockerhub_account  = var.dockerhub_account
-  instance_type      = "r5.large"
+  instance_type      = "r5.2xlarge"
 }
 
 // Serratus-dl
@@ -113,13 +113,13 @@ module "download" {
   source             = "../worker"
 
   desired_size       = 0
-  max_size           = 200
+  max_size           = 5000
 
   dev_cidrs          = var.dev_cidrs
   security_group_ids = [aws_security_group.internal.id]
 
-  instance_type  

In [24]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>



Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [22]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh /home/artem/serratus/notebook/200620_ab/v3_batch1.csv

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200620_ab/v3_batch1.csv
  date: Sat Jun 20 23:03:00 PDT 2020
  wc  : 300001 /home/artem/serratus/notebook/200620_ab/v3_batch1.csv
  md5 : 6b7c849958d432c81545927e2f1135bd  /home/artem/serratus/notebook/200620_ab/v3_batch1.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
ee031f609648928516382fdeb3d69caf  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":9999,"total_rows":9999}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
25568ff98b4e2da103f022c9ee9a691a  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":19999}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
3f6234b6bf3f01081612292c45a1f7ac  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":29999}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
e7917f2b305a6204959cb30fbf2e528b  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":3999

### Terraform Initialize  2

In [31]:
# For rapid batching; copy out serratus folder
SERRATUS=/home/artem/Desktop/run/serratus2
TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..a4fce76 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -92,7 +92,7 @@ module "scheduler" {
   
   security_group_ids = [aws_security_group.internal.id]
   key_name           = var.key_name
-  instance_type      = "c5.large"
+  instance_type      = "r5.4xlarge"
   dockerhub_account  = var.dockerhub_account
   scheduler_port     = var.scheduler_port
 }
@@ -105,7 +105,7 @@ module "monitoring" {
   key_name           = var.key_name
   scheduler_ip       = module.scheduler.private_ip
   dockerhub_account  = var.dockerhub_account
-  instance_type      = "r5.large"
+  instance_type      = "r5.2xlarge"
 }
 
 // Serratus-dl
@@ -113,13 +113,13 @@ module "download" {
   source             = "../worker"
 
   desired_size       = 0
-  max_size           = 200
+  max_size           = 5000
 
   dev_cidrs          = var.dev_cidrs
   security_group_ids = [aws_security_group.internal.id]
 
-  instance_type  

In [32]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>



Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [34]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh /home/artem/serratus/notebook/200620_ab/v3_batch2.csv

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200620_ab/v3_batch2.csv
  date: Sun Jun 21 03:44:31 PDT 2020
  wc  : 300001 /home/artem/serratus/notebook/200620_ab/v3_batch2.csv
  md5 : 30108519281553b6d488545bc4637796  /home/artem/serratus/notebook/200620_ab/v3_batch2.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
e75316982286fdfa64d63ed3b92010a7  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
3cbb5b0c4dc4b0b1c789e531465f2c41  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
335ca65f36494bc39291848bcc343d9f  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
ac3b7d3dc9e703bd42c3e0a4d77a1fb3  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":40

### Terraform Initialize  3

In [39]:
# For rapid batching; copy out serratus folder
SERRATUS=/home/artem/Desktop/run/serratus3
TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve



diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..a4fce76 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -92,7 +92,7 @@ module "scheduler" {
   
   security_group_ids = [aws_security_group.internal.id]
   key_name           = var.key_name
-  instance_type      = "c5.large"
+  instance_type      = "r5.4xlarge"
   dockerhub_account  = var.dockerhub_account
   scheduler_port     = var.scheduler_port
 }
@@ -105,7 +105,7 @@ module "monitoring" {
   key_name           = var.key_name
   scheduler_ip       = module.scheduler.private_ip
   dockerhub_account  = var.dockerhub_account
-  instance_type      = "r5.large"
+  instance_type      = "r5.2xlarge"
 }
 
 // Serratus-dl
@@ -113,13 +113,13 @@ module "download" {
   source             = "../worker"
 
   desired_size       = 0
-  max_size           = 200
+  max_size           = 5000
 
   dev_cidrs          = var.dev_cidrs
   security_group_ids = [aws_security_group.internal.id]
 
-  instance_type

In [40]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>



Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [42]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh /home/artem/serratus/notebook/200620_ab/v3_batch3.csv

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200620_ab/v3_batch3.csv
  date: Sun Jun 21 08:34:28 PDT 2020
  wc  : 300001 /home/artem/serratus/notebook/200620_ab/v3_batch3.csv
  md5 : c83cb5dc914218cd50f84ed59f7bf5af  /home/artem/serratus/notebook/200620_ab/v3_batch3.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
711995f171e0832dbb5db137ffd568e1  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
70308b7d6b011b9eae321270a82109d8  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
3e8650d2817b25fb170396b00e47956f  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
5d3bb3a233ae3881f6ea814e2f521763  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":40

### Terraform Initialize 4

In [48]:
# For rapid batching; copy out serratus folder
SERRATUS=/home/artem/Desktop/run/serratus4
TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..a4fce76 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -92,7 +92,7 @@ module "scheduler" {
   
   security_group_ids = [aws_security_group.internal.id]
   key_name           = var.key_name
-  instance_type      = "c5.large"
+  instance_type      = "r5.4xlarge"
   dockerhub_account  = var.dockerhub_account
   scheduler_port     = var.scheduler_port
 }
@@ -105,7 +105,7 @@ module "monitoring" {
   key_name           = var.key_name
   scheduler_ip       = module.scheduler.private_ip
   dockerhub_account  = var.dockerhub_account
-  instance_type      = "r5.large"
+  instance_type      = "r5.2xlarge"
 }
 
 // Serratus-dl
@@ -113,13 +113,13 @@ module "download" {
   source             = "../worker"
 
   desired_size       = 0
-  max_size           = 200
+  max_size           = 5000
 
   dev_cidrs          = var.dev_cidrs
   security_group_ids = [aws_security_group.internal.id]
 
-  instance_type  

In [49]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>



Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [51]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh /home/artem/serratus/notebook/200620_ab/v3_batch4.csv

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200620_ab/v3_batch4.csv
  date: Sun Jun 21 13:32:14 PDT 2020
  wc  : 300001 /home/artem/serratus/notebook/200620_ab/v3_batch4.csv
  md5 : 8df087d4fb2765755a86fe1a8855147a  /home/artem/serratus/notebook/200620_ab/v3_batch4.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
cf9370da50d3a9dfbbe3f3782bee072f  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
9c69ed9efa8722e0d1f4a2f56f0685f0  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
c41cc294891b9673af7dc66ef7b6b0f3  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
1ba5404590e7622359f33ffa31889996  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":40

### Terraform - Initialize 5

In [54]:
# For rapid batching; copy out serratus folder
SERRATUS=/home/artem/Desktop/run/serratus5
TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve



diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..a4fce76 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -92,7 +92,7 @@ module "scheduler" {
   
   security_group_ids = [aws_security_group.internal.id]
   key_name           = var.key_name
-  instance_type      = "c5.large"
+  instance_type      = "r5.4xlarge"
   dockerhub_account  = var.dockerhub_account
   scheduler_port     = var.scheduler_port
 }
@@ -105,7 +105,7 @@ module "monitoring" {
   key_name           = var.key_name
   scheduler_ip       = module.scheduler.private_ip
   dockerhub_account  = var.dockerhub_account
-  instance_type      = "r5.large"
+  instance_type      = "r5.2xlarge"
 }
 
 // Serratus-dl
@@ -113,13 +113,13 @@ module "download" {
   source             = "../worker"
 
   desired_size       = 0
-  max_size           = 200
+  max_size           = 5000
 
   dev_cidrs          = var.dev_cidrs
   security_group_ids = [aws_security_group.internal.id]
 
-  instance_type

In [55]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>



Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [59]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh /home/artem/serratus/notebook/200620_ab/v3_batch5.csv

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200620_ab/v3_batch5.csv
  date: Sun Jun 21 19:35:00 PDT 2020
  wc  : 155322 /home/artem/serratus/notebook/200620_ab/v3_batch5.csv
  md5 : 4db4460ebed0a6d7ec9d5edcc0989910  /home/artem/serratus/notebook/200620_ab/v3_batch5.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
361df29b82097e2b52fe677e8eb11e31  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
0dcc41b4197128244e7cfc086a8be469  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
452ba89b046ed2fd322991ec50be9896  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
1fc5f8aeb0300b1b70b0b736acf15b5b  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":40

In [61]:
cd $WORK
aws s3 ls s3://serratus-public/out/200620_inv/summary/ > v3.sra

cat v3.sra \
  | sed 's/^...............................//g' - \
  | cut -f1 -d'.' - > v3.sra.complete

wc -l v3.sra.complete # accidently overwrote previous v3.sra.complete

1112171 v3.sra.complete


In [62]:
# All runs missed on first pass
grep -vif v3.sra.complete v3_batch1.csv     | tail -n+2 - > v4_1.tmp
grep -vif v3.sra.complete v3_batch2.csv     | tail -n+2 - > v4_2.tmp
grep -vif v3.sra.complete v3_batch3.csv     | tail -n+2 - > v4_3.tmp
grep -vif v3.sra.complete v3_batch4.csv     | tail -n+2 - > v4_4.tmp

wc -l v4_*.tmp

cat sra.header v4_* > v4_SraRunInfo.csv

wc -l v4_SraRunInfo.csv
md5sum v4_SraRunInfo.csv

    7029 v4_1.tmp
    3803 v4_2.tmp
   21221 v4_3.tmp
   59318 v4_4.tmp
   91371 total
91372 v4_SraRunInfo.csv
a59f6c9231610dd7b8f5d12ec932ed65  v4_SraRunInfo.csv


In [63]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh /home/artem/serratus/notebook/200620_ab/v4_SraRunInfo.csv

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200620_ab/v4_SraRunInfo.csv
  date: Sun Jun 21 21:09:23 PDT 2020
  wc  : 91372 /home/artem/serratus/notebook/200620_ab/v4_SraRunInfo.csv
  md5 : a59f6c9231610dd7b8f5d12ec932ed65  /home/artem/serratus/notebook/200620_ab/v4_SraRunInfo.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
a313fbbee28f3dbbca25e53f33550f6a  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":9999,"total_rows":165319}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
9bec3d253b54d3fe016d328a8decb440  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":175319}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
570942f1a0a9b5b54bd7606752877e92  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":185319}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
3fbf028dff620f3e10047775084ad34e  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":9999,"to

## Run Serratus

In [67]:
# Set Cluster Parameters =============================
## get Config File (if it doesn't exist)
# curl localhost:8000/config | jq > serratus-config.json
#
cd $TF
# Make local changes to config file
echo "  Cluster Config File: "
cat serratus-config.json
echo ""
echo ""
# Re-upload config file
curl -T serratus-config.json localhost:8000/config

  Cluster Config File: 
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 50,
  "ALIGN_SCALING_CONSTANT": 0.025,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 0,
  "CLEAR_INTERVAL": 600000,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 15,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 0,
  "GENOME": "cov3ma",
  "MERGE_ARGS": "",
  "MERGE_MAX_INCREASE": 20,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 125,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 35
}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   541    0     0  100   541      0   2691 --:--:-- --:--:-- --:--:--  2678{"ALIGN_ARGS":"--very-sensitive-local","ALIGN_MAX_INCREASE":50,"ALIGN_SCALING_CONSTANT":0.025,"ALIGN_SCALING_ENABLE":true,"ALIGN_SCALING_MAX":0,"CLEAR_INTERVAL":600000,"DL_ARGS":"","DL_MAX_INCREASE

### Logging - batch 1
serratus-dl-2020062105583483840000000a
serratus-align-2020062105583498230000000c
serratus-merge-2020062105583485620000000b
tf-serratus-work-20200621055804310000000001

```
23:00 0 - - - 
00:00 24763 1275 1764 50
02:00 93669 38 3818 50 (downloads all done)
03:00 201581 0 74 250
03:35 290000 done
```

### Logging - batch 2

serratus-dl-20200621104205086900000005
serratus-align-20200621104205019200000004
serratus-merge-20200621104204918500000003

```
03:45 0 - - -
05:45 112300
08:20 296148
```

### Logging - batch 3
serratus-dl-2020062115315974820000000b
serratus-align-2020062115315975120000000c
serratus-merge-2020062115315974690000000a

```
8:40 0 - - -
9:40 34325 1300 2541 95
10:40 104093 1300 3319 95
11:00 127100 - - - 713,248 in 12 hours
11:40 172400 1300 3385 95
12:40 271300 - - - 
13:10 275635 - - -
```

### Logging - batch 4
138,217 remaining till 1 million

```
13:37 0 - - -
15:37 98600 - - -
16:18 1,000,000 total reached at 17:15:17.0
16:37 156200 - - - 
17:37 213500 - - -
19:13 239295
```

### Logging - batch 5

```
19:35 0 - - -
20:35 37000 - - -
21:35 97978 - - -
22:35 116000 1250 2750 85
23:03 183076 --> 1295247
23:50 232483
```

Total time: 24:50:22.2

### Error handling

In [ ]:
## Stop postgres if it's running 
# systemctl stop postgresql

## Connect to postgres
# psql -h localhost postgres postgres


#  psql -h localhost postgres postgres -c "DELETE FROM blocks WHERE state = 'done';"

### ACCESSION OPERATIONS
## Reset SPLITTING accessions to NEW
# UPDATE acc SET state = 'new' WHERE state = 'splitting';

## Reset SPLIT_ERR accessions to NEW
## (repeated failures can be missing SRA data)
# UPDATE acc SET state = 'new' WHERE state = 'split_err';

## Reset MERGE_ERR accessions to SPLIT_DONE
# UPDATE acc SET state = 'split_done' WHERE state = 'merge_err';

## Clear DONE Accessions (ONLY ON COMPLETION)
# DELETE FROM acc WHERE state = 'merge_done';

### BLOCK OPERATIONS

##  Reset FAIL blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'fail';

# Reset ALIGNING blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'aligning';

# Clear Done
# DELETE FROM blocks WHERE state = 'done';

# RESET STATE
# DELETE FROM blocks WHERE state = 'done';
# DELETE FROM blocks WHERE state = 'fail';
#
#
# DELETE FROM acc WHERE state = 'split_err';
# DELETE FROM acc WHERE state = 'merging';
# DELETE FROM acc WHERE state = 'merge_err';
# DELETE FROM acc WHERE state = 'split_done';


In [ ]:
aws autoscaling describe-auto-scaling-instances --max-items 10 \
  | jq -r '.AutoScalingInstances | .[] | .InstanceId' \
  > asg

# TERMINATE ALL DOWNLOADERS
aws autoscaling describe-auto-scaling-instances \
  | jq -r '.AutoScalingInstances | .[] | select(.AutoScalingGroupName=="serratus-dl-2020061502534816170000000c").InstanceId' \
  > asg2
  
for INSTANCE in $(cat asg2XX); do
  aws autoscaling terminate-instance-in-auto-scaling-group --instance-id $INSTANCE \
   --no-should-decrement-desired-capacity
done


# TERMINATE ALL ALIGNERS
aws autoscaling describe-auto-scaling-instances \
  | jq -r '.AutoScalingInstances | .[] | select(.AutoScalingGroupName=="serratus-align-2020061404510754780000000a").InstanceId' \
  > asg3

for INSTANCE in $(cat asg3XX); do
  aws autoscaling terminate-instance-in-auto-scaling-group --instance-id $INSTANCE \
   --no-should-decrement-desired-capacity
done
  
  
# TERMINATE ALL MERGE
aws autoscaling describe-auto-scaling-instances \
  | jq -r '.AutoScalingInstances | .[] | select(.AutoScalingGroupName=="serratus-merge-2020061404510767000000000c").InstanceId' \
  > asg

for INSTANCE in $(cat asg3XX); do
  aws autoscaling terminate-instance-in-auto-scaling-group --instance-id $INSTANCE \
   --no-should-decrement-desired-capacity
done




In [ ]:
aws autoscaling describe-auto-scaling-instances \
  | jq -r '.AutoScalingInstances | .[] | select(.AutoScalingGroupName=="serratus-align-2020061502534804080000000b").InstanceId' \
  > asg4

split -n 4 asg4

for INSTANCE in $(cat xaa); do
  aws autoscaling terminate-instance-in-auto-scaling-group --instance-id $INSTANCE \
   --no-should-decrement-desired-capacity
done

for INSTANCE in $(cat xab); do
  aws autoscaling terminate-instance-in-auto-scaling-group --instance-id $INSTANCE \
   --no-should-decrement-desired-capacity
done

for INSTANCE in $(cat xac); do
  aws autoscaling terminate-instance-in-auto-scaling-group --instance-id $INSTANCE \
   --no-should-decrement-desired-capacity
done

for INSTANCE in $(cat xad); do
  aws autoscaling terminate-instance-in-auto-scaling-group --instance-id $INSTANCE \
   --no-should-decrement-desired-capacity
done

In [ ]:
# Nuke Shutdown
aws ec2 describe-instances \
  --filter Name=tag:Name,Values=serratus-align-instance \
  > align_instances.json

jq '.Reservations[].Instances[].InstanceId' -r align_instances.json \
  | pv -l \
  | xargs -n10 -P10 aws ec2 terminate-instances --instance-ids

# Post-hoc cost analysis

Using Tag: Serratus + serratus.

The dates of June 21/22 have a total cost of (5798.82 + 1904.91) ÷ 1339925 accessions

= $0.0057 / accession
